## ASSIGNMENT-2
Roll Numbers:

Names:

### Instructions
 * Fill in the roll numbers and names of all the teams members in the cell above.
 * Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 * You can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
 * All the representations are expected to be in a right-hand coordinate system. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.

## G2O Motion Model 

Using the following motion model, you have to first generate the "initialization" for all the poses/vertices using the "Given" information. Just like in the 1D case.
$$x_{k+1} = x_{k} + \Delta x_{(k,k+1)} \cos(\theta_k) - \Delta y_{(k,k+1)} \sin(\theta_k) \\
y_{k+1} = y_{k} + \Delta y_{(k,k+1)} \cos(\theta_k) + \Delta x_{(k,k+1)} \sin(\theta_k) \\
\theta_{k+1} = \theta_{k}+  \Delta \theta_{(k,k+1)}$$

Even the loop closure nodes are related by the above model, except that it need not necessarily be consecutive nodes k and k+1.

## PART 2

### 1. Aligning Point Clouds using ICP (10 marks)

Given two 3D point clouds, P (`original_point_cloud.ply`) and Q (`transformed_point_cloud.ply`), where the second point cloud Q is a transformed version of the first point cloud P, apply the Iterative Closest Point (ICP) algorithm to align the two point clouds

Implement ICP using the Levenberg-Marquardt algorithm discussed in class. Compute the registration error and visualize the quality of alignment. 

Compare these results with the earlier ICP with SVD. Write a brief explanation on which one performs better (both time wise and accuracy wise), and why you think that is.

Correspondences are {P_i, Q_i} {i =  1 to N}

In [14]:
'''
YOUR CODE HERE
'''

'\nYOUR CODE HERE\n'

### 2. LieTorch (15 marks)
Resources: https://github.com/princeton-vl/lietorch/tree/master

The aim of this question is to use LieTorch for Pose Graph Optimization. You will be given a set of `n` noisy Transforms (representing your poses) and a set of `m` Loop Closure edges which will be generated by a function we have implemented. There are 2 parts for this question:
1. Keep the loop closure edges constant but adjust the `n` adjacent Transforms
2. Find a way to integrate confidence into the optimization (1 confidence value for the loop closure edges, and another for the adjacent edges), so that both the loop closure edges and the adjacent edges are optimized but to different degrees based on the confidence scores (Higher confidence implies that it's more likely that your edge is correct).

**Note:** Document your loss functions along with your implementation, with suitable explanation for the design choices you make. Also print your loss values, to show them converging.

Plot the original X,Y,Z values of your pose, their noisy counterparts and the new X,Y,Z values after optimization.

In [3]:
def getTransforms():
    '''
    Output Format - n, m, [i, j, x, y, z, q_x, q_y, q_z, q_w] * (m+n)
    The first n values will be the adjacent edges
    The last m values will be the loop closure edges

    '''
    pass


#### How you can mimic the getTransforms function:
Generate n random Transformations, create the loop closure edges between m of them at random, and then add noise to your original n Transformations.


In [4]:
'''
YOUR CODE HERE
'''

'\nYOUR CODE HERE\n'

### 3. Bonus (10 marks)

Implement the Pose Graph Optimization for the same G2O file as before using LieTorch, and compare the results against G2O using the Evo metrics.

You are not allowed to call any external python files for this question

Why is it performing better/worse?